In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 4.

dosave = False
savedir = "/icarus/app/users/gputnam/evtsel-ana/plots_Jan9/"

savedata = False
datadir = "/icarus/data/users/gputnam/dimuon-data/"

HMASS = 240
labelM = "$M_S$ = %iMeV" % HMASS
label = labelM + "\n$\\theta_S = 5\\cdot10^{-5}$"

# AMASS = 400
# labelM = "$M_a$ = %i MeV" % AMASS
# label = labelM + "\n$f_a$ = \n$c_\\ell$ ="

In [ ]:
fnu = "/icarus/data/users/gputnam/dimuon-data/mcnu_phase2_evt.df"

In [ ]:
nu_evtdf = pd.read_hdf(fnu, key="evt")

In [ ]:
nu_evtdf

In [ ]:
nu_mcdf = pd.read_hdf(fnu, key="mcnu")

In [ ]:
nu_mcdf

In [ ]:
nu_hdr = pd.read_hdf(fnu, key="hdr")

In [ ]:
nu_hdr

In [ ]:
fh = "/icarus/data/users/gputnam/dimuon-data/higgs_M%i_DMCP2023G_evt.df" % HMASS

In [ ]:
spectrum_file = fh.split("/")[-1].replace("evt.df", "spectrum.df").replace("DMCP2023G", "mcphase2")
spectrum_file

In [ ]:
h_evtdf = pd.read_hdf(fh, key="evt")

In [ ]:
h_mcdf = pd.read_hdf(fh, key="mcnu")

In [ ]:
h_mcdf

In [ ]:
h_mchdf = pd.read_hdf(fh, key="mch")

In [ ]:
h_mchdf

In [ ]:
h_hdr = pd.read_hdf(fh, key="hdr")

In [ ]:
h_hdr

In [ ]:
h_mchdf.index = h_mchdf.index.droplevel(-1)

In [ ]:
# MeVPrtl truth information

# Simulation parameters
p_M = h_mchdf.M
p_M.name = "p_M"

p_th = h_mchdf.C1
p_th.name = "p_th"

p_dist = dmagdf(h_mchdf.start, h_mchdf.enter)
p_dist.name = "p_dist"

p_raydist = dmagdf(h_mchdf.enter, h_mchdf.exit)
p_raydist.name = "p_raydist"

p_decaylength = h_mchdf.decay_length
p_decaylength.name = "p_decaylength"

h_evtdf[p_M.name] = p_M
h_evtdf[p_th.name] = p_th
h_evtdf[p_raydist.name] = p_raydist
h_evtdf[p_dist.name] = p_dist
h_evtdf[p_decaylength.name] = p_decaylength


nu_evtdf[p_M.name] = np.nan
nu_evtdf[p_th.name] = np.nan
nu_evtdf[p_dist.name] = np.nan
nu_evtdf[p_raydist.name] = np.nan
nu_evtdf[p_dist.name] = p_raydist
nu_evtdf[p_decaylength.name] = np.nan

In [ ]:
def sample_concat(dfA, dfB, offset=1_000_000):
    dfB = dfB.copy()
    # dfB.index = dfB.index.set_levels(dfB.index.levels[0] + dfA.index.max()[0] + 1, level=0)
    dfB.index = dfB.index.set_levels(dfB.index.levels[0] + offset, level=0)
    return dfA.append(dfB)

In [ ]:
nu_pot = np.sum(nu_hdr.pot * nu_hdr.first_in_subrun)
h_pot = np.sum(h_hdr.pot * h_hdr.first_in_subrun)

In [ ]:
nu_evt_per_spill = np.sum(nu_hdr.ngenevt*nu_hdr.first_in_subrun) / (np.sum(nu_hdr.pot*nu_hdr.first_in_subrun) / 6e13)
# ic_pot = np.sum(ic_hdr.ngenevt * ic_hdr.first_in_subrun * 6e13) / (1 - nu_evt_per_spill)

In [ ]:
GOAL_POT = 4e20 
nu_evtdf["scale"] = GOAL_POT / nu_pot
h_evtdf["scale"] = GOAL_POT / h_pot
# ic_evtdf["scale"] = GOAL_POT / ic_pot

nu_mcdf["scale"] = GOAL_POT / nu_pot
h_mcdf["scale"] = GOAL_POT / h_pot

POTSTR = "4e20 POT"

In [ ]:
h_evtdf["higgs"] = True
nu_evtdf["higgs"] = False
# ic_evtdf["higgs"] = False

In [ ]:
def decay_weight(in_dist, out_dist, mean_dist):
    return np.exp(-in_dist / mean_dist) - \
          np.exp(-out_dist / mean_dist)
    
def flux_weight(mixing):
    return mixing * mixing

def reweight_mixing(newmixing, start, enter, exit, mean_dist, oldmixing=1e-5):
    dist_in = dist(enter, start)
    dist_out = dist(exit, start)
    
    old_dcy = decay_weight(dist_in, dist_out, mean_dist)
    new_dcy = decay_weight(dist_in, dist_out, mean_dist / (newmixing**2 / oldmixing**2))
    
    old_flux = flux_weight(oldmixing)
    new_flux = flux_weight(newmixing)
    
    return (new_flux / old_flux) * (new_dcy / old_dcy)

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 5, 21)

_ = plt.hist([var[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],
              var[when & (nu_mcdf.np == 0) & ((nu_mcdf.genie_mode != 3) | (nu_mcdf.iscc == 0))],
              var[when & (nu_mcdf.np > 0)],
             ],
              weights=[
                  nu_mcdf.scale[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],
                  nu_mcdf.scale[when & (nu_mcdf.np == 0) & ((nu_mcdf.genie_mode != 3) | (nu_mcdf.iscc == 0))],
                  nu_mcdf.scale[when & (nu_mcdf.np > 0)]
              ],
              bins=bins, label=["CC Coh", "No p", "With p"],
             stacked=True
            )
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy_splitnu.pdf")

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 10, 21)

_ = plt.hist([var[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],],
              weights=[nu_mcdf.scale[when & (nu_mcdf.genie_mode == 3) & nu_mcdf.iscc],],
              bins=bins, label=["CC Coh"],
             stacked=True
            )
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy_cohnu.pdf")

In [ ]:
var = nu_mcdf.E
when = SlcInFV(nu_mcdf.position)
bins = np.linspace(0, 5, 21)

_ = plt.hist(var[when], bins=bins, weights=nu_mcdf.scale[when])
plt.ylabel("Fiducial Neutrinos / %s" % POTSTR)
plt.xlabel("Neutrino Energy [GeV]")
plt.tight_layout()

if dosave: plt.savefig(savedir + "neutrino_energy.pdf")

In [ ]:
var = h_mcdf.E
when = SlcInFV(h_mcdf.position)
bins = np.linspace(0, 5, 21)
bins = np.linspace(0, 10, 21)

_ = plt.hist(var[when], bins=bins, weights=h_mcdf.scale[when])
plt.ylabel("Fiducial Scalars / %s" % POTSTR)
plt.xlabel("Scalar Energy [GeV]")
plt.tight_layout()
plt.text(0.6, 0.6, label, fontsize=18, transform=plt.gca().transAxes)
if dosave: plt.savefig(savedir + "scalar_energy_M%i.pdf" % HMASS)

In [ ]:
numu_cc = (np.abs(nu_mcdf.pdg) == 14) & nu_mcdf.iscc

mc_is_nu_coh = (nu_mcdf.genie_mode == 3) & numu_cc
mc_is_nu_coh.name = "$\\nu_\\mu$ CC COH"

mc_is_nu_pi = (nu_mcdf.npi >= 1) & (nu_mcdf.max_proton_ke < 0.02) & (nu_mcdf.npi0 == 0) & ~mc_is_nu_coh & numu_cc
mc_is_nu_pi.name = "$\\nu_\\mu$ CC n$\\pi$0p"
                                              
mc_is_nu_pip = (nu_mcdf.npi >= 1) & (nu_mcdf.max_proton_ke >= 0.02) & (nu_mcdf.npi0 == 0) & ~mc_is_nu_coh & numu_cc
mc_is_nu_pip.name = "$\\nu_\\mu$ CC n$\\pi$np"                                              

mc_is_nu_other = ~mc_is_nu_coh & ~mc_is_nu_pi & ~mc_is_nu_pip & numu_cc
mc_is_nu_other.name = "$\\nu_\\mu$ CC Other"

In [ ]:
mc_nucategories = [mc_is_nu_other, mc_is_nu_pip, mc_is_nu_pi, mc_is_nu_coh] #[mc_is_nu_pi, mc_is_nu_pip, mc_is_nu_coh, mc_is_nu_other]

In [ ]:
modes = [0, 1, 2, 3, 10]
mode_names = ["QE", "RES", "DIS", "COH", "MEC"]

In [ ]:
var = nu_mcdf.E

when = SlcInFV(nu_mcdf.position) & numu_cc

bins = np.linspace(0, 10, 21)

# for c in nucategories:
_ = plt.hist([var[when & (nu_mcdf.genie_mode == m)] for m in modes], 
             weights=[nu_mcdf.scale[when & (nu_mcdf.genie_mode == m)]/10 for m in modes], bins=bins, 
             label=mode_names, 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Neutrino Energy [GeV]")
plt.title("ICARUS MC")
plt.ylabel("Neutrinos / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "all_nue_intmodes.pdf")

In [ ]:
evtdf = sample_concat(h_evtdf, nu_evtdf, offset=1_000_000)

In [ ]:
is_higgs = (evtdf.slc.tmatch.idx >= 0) & evtdf.higgs & (evtdf.slc.truth.npi == 0) & (evtdf.slc.truth.npi0 == 0)
is_higgs.name = "Scalar"
is_nu = (evtdf.slc.tmatch.idx >= 0) & ~evtdf.higgs
is_nu.name = "$\\nu$"
is_cosmic = evtdf.slc.tmatch.idx < 0
is_cosmic.name = "Cosmic"

In [ ]:
categories = [is_higgs, is_nu, is_cosmic]

In [ ]:
evt_numu_cc = (np.abs(evtdf.slc.truth.pdg) == 14) & evtdf.slc.truth.iscc

is_nu_coh = is_nu & (evtdf.slc.truth.genie_mode == 3) & (evtdf.slc.truth.iscc) & evt_numu_cc
is_nu_coh.name = "$\\nu_\\mu$ CC COH"

is_nu_pi = is_nu & (evtdf.slc.truth.npi >= 1) & (evtdf.slc.truth.max_proton_ke < 0.02) & (evtdf.slc.truth.npi0 == 0) & (evtdf.slc.truth.iscc) & evt_numu_cc & ~is_nu_coh
is_nu_pi.name = "$\\nu_\\mu$ CC n$\\pi$0p"
                                              
is_nu_pip = is_nu & (evtdf.slc.truth.npi >= 1) & (evtdf.slc.truth.max_proton_ke >= 0.02) & (evtdf.slc.truth.npi0 == 0) & (evtdf.slc.truth.iscc) & evt_numu_cc & ~is_nu_coh
is_nu_pip.name = "$\\nu_\\mu$ CC n$\\pi$np"                                              

is_nu_other = is_nu  & ~is_nu_coh & ~is_nu_pi & ~is_nu_pip
is_nu_other.name = "$\\nu$ Other"

In [ ]:
nucategories = [is_nu_other, is_nu_pip, is_nu_pi, is_nu_coh]

In [ ]:
scale = evtdf.scale.copy()
# scale[is_higgs] = evtdf.scale

In [ ]:
var = np.maximum(evtdf.trunk.trk.len, evtdf.branch.trk.len)

when = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.is_clear_cosmic == 0)

bins = np.linspace(0, 400, 21)

# for c in nucategories:
_ = plt.hist([var[when & c] for c in [is_cosmic] + nucategories], 
             weights=[scale[when & c]/10 for c in [is_cosmic] + nucategories], bins=bins, 
             label=[c.name for c in [is_cosmic] + nucategories], 
                 stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Maximum Track Length [cm]")
plt.title("ICARUS MC")
plt.ylabel("Candidates / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "allevt_trklen.pdf")

In [ ]:
# DEFINE CUTS

In [ ]:
# OPTIMIZED
MAX_SHW_LEN_CUT = 3
OTHR_CHI2P_CUT = 80
OTHR_CHI2U_CUT = 40

LEN_CUT = 25

CHI2U_CUT = 10
# CHI2U_CUT = 6
CHI2P_CUT = 80
MED_DQDX_CUT = 0

STUB_5mm_CUT = 5e5 / 1e3
STUB_10mm_CUT = 3e5 / 1e3
STUB_20mm_CUT = 2.5e5 / 1e3
STUB_30mm_CUT = 2e5 / 1e3

OPENANGLE_CUT = 0.5
UU_ANGLE_CUT = 0.125

In [ ]:
# LOOSE
# MAX_SHW_LEN_CUT = 10
# OTHR_CHI2P_CUT = 80
# OTHR_CHI2U_CUT = 40
# LEN_CUT = 25

# CHI2U_CUT = 20
# CHI2P_CUT = 80
# MED_DQDX_CUT = 0

# STUB_5mm_CUT = 5e5 / 1e3
# STUB_10mm_CUT = 3e5 / 1e3
# STUB_20mm_CUT = 2.5e5 / 1e3
# STUB_30mm_CUT = 2e5 / 1e3

# OPENANGLE_CUT = 0.6
# UU_ANGLE_CUT = 0.15

In [ ]:
when = SlcInFV(evtdf.slc.vertex)

var = evtdf.max_shw_len.copy()
var[np.isnan(var) | (var < 0)] = -10

bins = [-12, -8] + list(np.linspace(0, 100, 26))

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.xlabel("Max Shower Length [cm]")

plt.ylabel("Area Normalized")
plt.yscale("log")

plt.axvline([MAX_SHW_LEN_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()

if dosave: plt.savefig(savedir + "max_shw_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)

var = evtdf.third_trk_dist.copy()
var[np.isnan(var) | (var < 0)] = -10

bins = [-12, -8] + list(np.linspace(0, 15, 16))

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.xlabel("Third Track Dist [cm]")

plt.ylabel("Area Normalized")
plt.yscale("log")

# plt.axvline([MAX_SHW_LEN_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()

if dosave: plt.savefig(savedir + "third_trk_len_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) 
var = evtdf.min_othr_chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([OTHR_CHI2P_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"thirdtrk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex)
var = evtdf.max_othr_chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([OTHR_CHI2U_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_muon_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end)
var = evtdf.trunk.trk.chi2pid.I2.chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([CHI2U_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_muon_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end)
var = evtdf.trunk.trk.chi2pid.I2.chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([CHI2P_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end)
var = evtdf.trunk.trk.len

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Len [cm] (Contained)")

plt.ylabel("Area Normalized")
plt.tight_layout()
#nplt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_cont_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & ~TrkInFV(evtdf.trunk.trk.end)
var = evtdf.trunk.trk.len

bins = np.linspace(0, 600, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()

plt.xlabel("Trunk Len [cm] (Exiting)")

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.axvline([60], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir +"trunk_cont_len_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.trk.end)
var = evtdf.branch.trk.chi2pid.I2.chi2_muon

bins = np.linspace(0, 80, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Muon (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([CHI2U_CUT], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "branch_chi2_muon_M%i.pdf" % HMASS)

In [ ]:
when = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.branch.trk.end)
var = evtdf.branch.trk.chi2pid.I2.chi2_proton

bins = np.linspace(0, 400, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", density=True, linewidth=2)
plt.legend()

plt.xlabel("Branch Track Col. Plane $\\chi^2$ Proton (Contained)")

plt.ylabel("Area Normalized")
plt.axvline([CHI2P_CUT], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.text(0.6, 0.4, label, transform=plt.gca().transAxes, fontsize=18)

if dosave: plt.savefig(savedir + "branch_chi2_proton_M%i.pdf" % HMASS)

In [ ]:
def is_muon(trk):
    return (TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon < CHI2U_CUT) & (trk.chi2pid.I2.chi2_proton > CHI2P_CUT)) | (~TrkInFV(trk.end) & (trk.len > 100.))

def is_proton(trk):
    return TrkInFV(trk.end) & (trk.chi2pid.I2.chi2_muon > 40) & (trk.chi2pid.I2.chi2_proton < 60)

In [ ]:
muselect = is_muon(evtdf.trunk.trk) & is_muon(evtdf.branch.trk)
muselect.name = "2 Muon Tracks Reconstructed"

In [ ]:
LAr_density_gmL = 1.389875

# RECOMBINATION

# ArgoNeuT params
MODA = 0.930
MODB = 0.212
Wion = 1e3 / 4.237e7

Vps = 75058
Enom = (375./385)*Vps*1e-3 /148.25
Eshort = (368.6/378.6)*Vps*1e-3 /148.25
Efield = Enom

def recombination(dEdx, A=MODA, B=MODB, E=Efield):
    alpha = A
    beta = B / (LAr_density_gmL * E)

    dQdx = np.log(alpha + dEdx*beta) / (Wion * beta)
    return dQdx

# recombination correction factor, MC
Rfactor = 1/(recombination(1.6, A=0.93, B=0.212, E=0.494)*Wion)
Gain = 1/0.01265
Rfactor

In [ ]:
branch2trunk = magdf(evtdf.trunk.trk.start - evtdf.branch.trk.start)

when = SlcInFV(evtdf.slc.vertex) & (branch2trunk < 5) #& topselect & muselect & (branch2trunk > 5)
var = evtdf.trunk.med_start_dqdx*Gain*Rfactor / 1e3

bins = np.linspace(0, 2e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([MED_DQDX_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.5, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")

plt.xlabel("Trunk dQ/dx [k$e^-$/cm]")
plt.ticklabel_format(axis='y', style='sci', scilimits=(-1,-4))

plt.tight_layout()

if dosave: plt.savefig(savedir + "trunk_med_dqdx_M%i.pdf" % HMASS)

In [ ]:
branch2trunk = magdf(evtdf.trunk.trk.start - evtdf.branch.trk.start)

when = SlcInFV(evtdf.slc.vertex) & (branch2trunk > 5) #& topselect & muselect & (branch2trunk > 5)
var = evtdf.trunk.med_start_dqdx*Gain*Rfactor / 1e3

bins = np.linspace(0, 2e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([MED_DQDX_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.6, 0.5, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")

plt.xlabel("Trunk dQ/dx [ADDC/cm]")
plt.ticklabel_format(axis='y', style='sci', scilimits=(-1,-4))

plt.tight_layout()

if dosave: plt.savefig(savedir + "trunk_med_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = np.arccos(dotdf(evtdf.trunk.trk.dir, evtdf.branch.trk.dir)) #* 180 / np.pi
when = SlcInFV(evtdf.slc.vertex) #& topselect & muselect

bins = np.linspace(0, np.pi, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, histtype="step", linewidth=2, density=True)
plt.legend(loc="upper right")

plt.axvline([OPENANGLE_CUT], linestyle=":", color="r", linewidth=2)

plt.text(0.25, 0.75, label, transform=plt.gca().transAxes, fontsize=18)

plt.ylabel("Area Normalized")
plt.tight_layout()
plt.xlabel("Reconstructed Opening Angle")

if dosave: plt.savefig(savedir + "openangle_M%i.pdf" % HMASS)

In [ ]:
beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 
BEAMDIR

In [ ]:
def recop(trk):
    p = trk.rangeP.p_muon
    p[~TrkInFV(trk.end)] = trk.mcsP.fwdP_muon
    return p

In [ ]:
scalar_mom = evtdf.trunk.trk.dir.multiply(recop(evtdf.trunk.trk), axis=0) + \
           evtdf.branch.trk.dir.multiply(recop(evtdf.branch.trk), axis=0)
    
scalar_dir = scalar_mom.divide(magdf(scalar_mom), axis=0)

beamangle = np.arccos(scalar_dir.x*BEAMDIR[0] + scalar_dir.y*BEAMDIR[1] + scalar_dir.z*BEAMDIR[2])

trunk_beamcos = evtdf.trunk.trk.dir.x*BEAMDIR[0] + evtdf.trunk.trk.dir.y*BEAMDIR[1] + evtdf.trunk.trk.dir.z*BEAMDIR[2]
trunk_E = np.sqrt(recop(evtdf.trunk.trk)**2 + MUON_MASS**2)
branch_beamcos = evtdf.branch.trk.dir.x*BEAMDIR[0] + evtdf.branch.trk.dir.y*BEAMDIR[1] + evtdf.branch.trk.dir.z*BEAMDIR[2]
branch_E = np.sqrt(recop(evtdf.branch.trk)**2 + PION_MASS**2)


coh_t = (np.sin(beamangle)*magdf(scalar_mom))**2 + (trunk_E - recop(evtdf.trunk.trk)*trunk_beamcos + branch_E - recop(evtdf.branch.trk)*branch_beamcos)**2

In [ ]:
var = beamangle

when = SlcInFV(evtdf.slc.vertex) & muselect & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

bins = np.linspace(0, 0.5, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([UU_ANGLE_CUT], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Reconstructed Scalar Angle to Beam [rad]")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.45, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_beam_angle_relativenorm_M%i.pdf" % HMASS)

In [ ]:
var = np.abs((recop(evtdf.trunk.trk) - recop(evtdf.branch.trk)) / (recop(evtdf.trunk.trk) + recop(evtdf.branch.trk)))

when = SlcInFV(evtdf.slc.vertex) & muselect

bins = np.linspace(0, 1, 21)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend(loc="lower left")
# plt.axvline([0.3], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Momentum Assymetry")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.4, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "momentum_balance_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l0_5cm.dqdx / 1e3

when = SlcInFV(evtdf.slc.vertex) & muselect # & (~is_higgs | ((evtdf.trunk.truth.p.genE > 0.5) & (evtdf.branch.truth.p.genE > 0.5)))

bins = np.linspace(0, 8e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([STUB_5mm_CUT], linestyle=":", color="r", linewidth=2)

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Stub dQ/dx [k$e^-$/cm] (len < 0.5cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.25, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_0_5cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l1cm.dqdx / 1e3

bins = np.linspace(0, 8e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([STUB_10mm_CUT], linestyle=":", color="r", linewidth=2)

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Stub dQ/dx [k$e^-$/cm] (0.5 < len < 1cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_0_5cm_1cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l2cm.dqdx / 1e3

bins = np.linspace(0, 8e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
plt.legend()
plt.axvline([STUB_20mm_CUT], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Stub dQ/dx [k$e^-$/cm] (1cm < len < 2cm)")
plt.title("ICARUS MC")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)
plt.ylabel("Area Normalized")

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_1cm_2cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.stub.l3cm.dqdx / 1e3

bins = np.linspace(0, 8e5, 21) / 1e3

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2, density=True)
    
plt.legend()
plt.axvline([STUB_30mm_CUT], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Stub dQ/dx [k$e^-$/cm] (2cm < len < 3cm)")
plt.title("ICARUS MC")
plt.ylabel("Area Normalized")

plt.text(0.625, 0.35, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "stub_2cm_3cm_dqdx_M%i.pdf" % HMASS)

In [ ]:
hasstub = (evtdf.stub.l0_5cm.dqdx/1e3 > STUB_5mm_CUT) |\
          (evtdf.stub.l1cm.dqdx/1e3 > STUB_10mm_CUT) |\
          (evtdf.stub.l2cm.dqdx/1e3 > STUB_20mm_CUT) |\
          (evtdf.stub.l3cm.dqdx/1e3 > STUB_30mm_CUT)

In [ ]:
preselect = [
    SlcInFV(evtdf.slc.vertex),
#     evtdf.slc.nuid.crlongtrkdiry > -0.7,
    np.isnan(evtdf.max_shw_len) | (evtdf.max_shw_len < MAX_SHW_LEN_CUT),
    np.isnan(evtdf.max_othr_chi2_muon) | (evtdf.max_othr_chi2_muon < OTHR_CHI2U_CUT),
    np.isnan(evtdf.min_othr_chi2_proton) | (evtdf.min_othr_chi2_proton > OTHR_CHI2P_CUT),
]

muid = [
    evtdf.trunk.trk.len > LEN_CUT,
    evtdf.branch.trk.len > LEN_CUT,
    is_muon(evtdf.trunk.trk),
    is_muon(evtdf.branch.trk),
    (evtdf.trunk.med_start_dqdx*Gain*Rfactor/1e3 > MED_DQDX_CUT) | np.isnan(evtdf.trunk.med_start_dqdx) | (magdf(evtdf.trunk.trk.start - evtdf.branch.trk.start) < 5),    
]

kinematics = [
    np.arccos(dotdf(evtdf.trunk.trk.dir, evtdf.branch.trk.dir)) < OPENANGLE_CUT,
    beamangle < UU_ANGLE_CUT,
]

stubs = [
    ~hasstub,
]

def cut_all(l):
    l0 = l[0]
    for c in l[1:]:
        l0 = l0 & c
    return l0

# cuts = preselect + objects + scores + muid + stubs + kinematics
cuts = preselect + muid + stubs + kinematics

passcut = cut_all(cuts)

passcut_nokinematics = cut_all(preselect + muid + stubs + [kinematics[0]])

cutlist = [preselect, muid, stubs, kinematics]
cutlist_names = ["Pre-Select", "2$\\mu$ ID", "Stub", "Kinematics", ]

In [ ]:
var = coh_t

when = SlcInFV(evtdf.slc.vertex) & passcut_nokinematics

bins = np.linspace(0, 0.2, 11)

_ = plt.hist(
    [var[when & is_nu_coh], var[when & is_nu & ~is_nu_coh]],
    weights=[scale[when & is_nu_coh], scale[when & is_nu & ~is_nu_coh]],
    bins=bins, label=["$\\nu$ Coh-$\pi$", "$\\nu$ Incoh."],
    stacked=True
)
    
plt.legend()
# plt.axvline([0.15], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("$|t|$ [Gev$^2$]")
plt.title("ICARUS MC")
plt.ylabel("Events / %s" % POTSTR)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_beam_angle_relativenorm_M%i.pdf" % HMASS)

In [ ]:
var = beamangle

when = SlcInFV(evtdf.slc.vertex) & passcut_nokinematics

bins = np.linspace(0, 0.4, 9)

_ = plt.hist(
    [var[when & is_nu_coh], var[when & is_nu & ~is_nu_coh]],
    weights=[scale[when & is_nu_coh], scale[when & is_nu & ~is_nu_coh]],
    bins=bins, label=["$\\nu$ Coh-$\pi$", "$\\nu$ Incoh."],
    stacked=True
)
    
plt.legend()
# plt.axvline([0.15], linestyle=":", color="r", linewidth=2)

plt.tight_layout()
plt.xlabel("Reconstructed Scalar Angle to Beam [rad]")
plt.title("ICARUS MC")
plt.ylabel("Events / %s" % POTSTR)
#plt.text("<- Signal", color=)
plt.axvline([0.15], color="black", linestyle=":")
plt.text(0.15, 0.6, "<- Signal", color="black", transform=plt.gca().transAxes)
plt.text(0.4, 0.4, "Constraint ->", color="black", transform=plt.gca().transAxes)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_beam_angle_relativenorm_M%i.pdf" % HMASS)

In [ ]:
Etrunk = np.sqrt(MUON_MASS**2 + recop(evtdf.trunk.trk)**2)
Ebranch = np.sqrt(MUON_MASS**2 + recop(evtdf.branch.trk)**2)

s_mass = np.sqrt(2)*np.sqrt(MUON_MASS**2 + Etrunk*Ebranch - recop(evtdf.trunk.trk)*recop(evtdf.branch.trk)*dotdf(evtdf.branch.trk.dir, evtdf.trunk.trk.dir))

In [ ]:
plt.figure(figsize=(9, 6))

bins = np.linspace(0, 10, 21)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & (evtdf.slc.tmatch.eff > 0.5)
# slc_is_select = slc_is_reco & passcut

thiscut = True
for c, name in zip(cutlist, cutlist_names):
    thiscut = cut_all(c) & thiscut
    when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
    D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.65, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % HMASS)

In [ ]:
plt.figure(figsize=(9, 6))

bins = np.linspace(0, 10, 21)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)
# slc_is_select = slc_is_reco & passcut

thiscut = True
for c, name in zip(cutlist, cutlist_names):
    thiscut = cut_all(c) & thiscut
    when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
    D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.65, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % HMASS)

In [ ]:
plt.figure(figsize=(9, 6))

bins = np.linspace(0, 10, 21)

when_all = SlcInFV(h_mcdf.position) & (h_mcdf.npi == 0) & (h_mcdf.npi0 == 0)

slc_is_reco = SlcInFV(evtdf.slc.vertex) & (evtdf.slc.tmatch.idx == 0) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

thiscut = True
for c, name in zip(cutlist, cutlist_names):
    thiscut = cut_all(c) & thiscut
    when_cut = when_all & (slc_is_reco & thiscut)[is_higgs].groupby(level=[0, 1]).any()
    D, _ = np.histogram(h_mcdf[when_all].E, bins=bins)
    N, _ = np.histogram(h_mcdf[when_cut].E, bins=bins)  

    centers = (bins[1:] + bins[:-1])/2.

    plt.plot(centers, N/D, label=name)
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")

plt.xlabel("Scalar Energy [GeV]")
plt.ylabel("Efficiency")
plt.text(0.65, 0.05, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "scalar_eff_all_M%i.pdf" % HMASS)

In [ ]:
width = 0.25
plt.figure(figsize=(6.4, 6.4))

basecut = SlcInFV(evtdf.slc.vertex) & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

for i,category in enumerate(categories):
    plt.barh(np.arange(len(cutlist)) + i*width, 
             [np.sum(evtdf.scale[category & basecut & cut_all([c for cuts in cutlist[:i+1] for c in cuts])]) for i in range(len(cutlist))], 
             width, 
             label=category.name,
             tick_label = cutlist_names
            )
    plt.xscale("log")
    plt.legend(ncol=3, bbox_to_anchor=(-0.05, 1.0, 1, 0.2), loc="upper left")
    plt.xlabel("Scalar Candidates / %s (1yr)" % POTSTR)
plt.xlim([1, plt.xlim()[1]])
plt.title(label.replace("\n", ", "))
plt.tight_layout()
if dosave: plt.savefig(savedir + "cutchart_M%i.pdf" % HMASS)

In [ ]:
var = evtdf.slc.truth.max_proton_ke*1e3

when_cut = SlcInFV(evtdf.slc.vertex) & passcut

bins = np.linspace(0, 0.2, 21)*1e3

for c in [is_nu]: 
    _ = plt.hist(var[when_cut & c], weights=scale[when_cut & c], bins=bins, label=c.name, histtype="step", linewidth=2) 

when_cut_nostub = SlcInFV(evtdf.slc.vertex) & cut_all(preselect + muid + kinematics)

for c in [is_nu]: 
    _ = plt.hist(var[when_cut_nostub & c], weights=scale[when_cut_nostub & c], bins=bins, label=c.name, histtype="step", linewidth=2) 

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4)) plt.tight_layout() plt.xlabel("Max Proton KE [GeV]") plt.title("ICARUS MC") plt.ylabel("Candidates / 6e20 POT")

plt.tight_layout() 
if dosave: plt.savefig(savedir + "passcut_proton_ke.pdf")

In [ ]:
var = s_mass

when = SlcInFV(evtdf.slc.vertex) & passcut & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

bins = np.linspace(0.2, 0.8, 25)

Ns = [scale[when & (evtdf.slc.truth.genie_mode == m)].sum() for m in modes]

_ = plt.hist([var[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], 
             weights=[scale[when & (evtdf.slc.truth.genie_mode == m)] for m in modes], bins=bins, 
             label=[n + " (%.0f%%)" % (N/np.sum(Ns)*100) for n,N in zip(mode_names, Ns)], 
            stacked=True, linewidth=2)
plt.legend()

#plt.ticklabel_format(axis='y', style='sci', scilimits=(4,4))
plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.tight_layout()
if dosave: plt.savefig(savedir + "smass_intmodes.pdf")

In [ ]:
var = s_mass
bins = np.linspace(0.2, 0.8, 61)

when = SlcInFV(evtdf.slc.vertex) & passcut & TrkInFV(evtdf.trunk.trk.end) & TrkInFV(evtdf.branch.trk.end)

for c in categories:
    _ = plt.hist(var[when & c], weights=scale[when & c], bins=bins, label=c.name, 
                 histtype="step", linewidth=2)
plt.legend()

plt.tight_layout()
plt.xlabel("Scalar Mass [GeV/c$^2$]")
plt.ylabel("Events / %s" % POTSTR)
plt.title("ICARUS MC")

plt.text(0.625, 0.325, label, transform=plt.gca().transAxes, fontsize=18)

plt.tight_layout()
if dosave: plt.savefig(savedir + "passcut_smass_M%i_contained.pdf" % HMASS)

In [ ]:
# make a dataframe with just the information neede for the spectrum

In [ ]:
spectradf = pd.DataFrame(
    data={
        "mass": s_mass[passcut_nokinematics],
        "thnumi": beamangle[passcut_nokinematics], 
        "tuu": coh_t[passcut_nokinematics],
        "signal": evtdf.higgs[passcut_nokinematics],
        "weight": scale[passcut_nokinematics],
        "genie_mode": evtdf.slc.truth.genie_mode[passcut_nokinematics],
        "p_M": evtdf.p_M[passcut_nokinematics],
        "p_th": evtdf.p_th[passcut_nokinematics],
        "p_dist": evtdf.p_dist[passcut_nokinematics],
        "p_raydist": evtdf.p_raydist[passcut_nokinematics],
        "p_decaylength": evtdf.p_decaylength[passcut_nokinematics],
        
    },
).reset_index(drop=True)

In [ ]:
spectradf[spectradf.signal]

In [ ]:
if savedata:
    spectradf.to_hdf(datadir + spectrum_file, key="s")

In [ ]:
spectrum_file